<img align="right" src="../../additional_data/banner_siegel.png" style="width:1000px;">

# Advanced Xarray

* [**Sign up to the JupyterHub**](https://www.phenocube.org/) to run this notebook interactively from your browser
* **Compatibility:** Notebook currently compatible with the Open Data Cube environments of the University of Wuerzburg
* **Products used**: `Sentinel-2`
* **Prerequisites**:  Users of this notebook should have a basic understanding of:
    * How to run a [Jupyter notebook](01_jupyter_introduction.ipynb)
    * The basic structure of the [eo2cube satellite datasets](02_eo2cube_introduction.ipynb)
    * How to [browse through the available products/measurements and how to load data from the eo2cube datacube](03_data_lookup_and_loading.ipynb)
    * How the data is stored and structured in a [xarray](04_xarrayI_data_structure.ipynb)

## Background

The Python library `xarray` simplifies working with labeled multi-dimension arrays. The library introduces labels in the forms of dimensions, coordinates, and attributes on top of `NumPy` arrays. This structure allows easier and more effective handling of remote sensing raster data in a Python environment. Therefore, it is essential to understand the structure of a `xarray` fully. 

The last notebook  ["04_xarrayI_data_structure.ipynb"](04_xarrayI_data_structure.ipynb) gave a first introduction to the usage of `xarray` within the eo2cube environment. 
This notebook builds on this gained knowledge and attempts to give a deeper understanding of the `xarray` data structure of raster data.
To get more information about the `xarray` package, visit the [offical documentation website](http://xarray.pydata.org/en/stable/).

## Description

This notebook introduces users to the `xarray` library within the datacube environment. It aims to deepen the understanding of the `xarray` structure as a container for remote sensing raster data. Also, it introduces useful `xarray` functions to effectivly work with raster data in the eo2cube environment. 

Within this notebook, the following topics are covered:

* Application of built-in `xarray` functions for analyzing raster data

***

## **Setting up**
### Load packages

The `datacube` package is required to query the eo2cube datacube database and load the requested data. The `with_ui_cbk` function from `odc.ui` enables a progress bar when loading large amounts of data. The `xarray` and `NumPy` package are needed for this notebook's different methods and analysis steps.

In [1]:
import datacube
from odc.ui import with_ui_cbk
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

dc = datacube.Datacube(app = "nb_understand_ndArrays")

### Datacube connection and load data

First, we connect to the datacube and load an example dataset from the eo2cube.

In [2]:
# Load Data Product
ds = dc.load(product= "s2_l2a",
             x = (24.78 ,24.88),
             y = (-28.90, -28.81),
             output_crs = "EPSG:32734",
             time = ("2020-10-01", "2020-12-31"),
             measurements= ["blue", "green", "red","nir"],
             resolution = (-10,10),
             group_by = "solar_day",
             progress_cbk=with_ui_cbk())

ds

/opt/conda/lib/python3.9/site-packages/datacube/drivers/postgres/_api.py:564: SAWarning: Class CommonTimestamp will not make use of SQL compilation caching as it does not set the 'inherit_cache' attribute to ``True``.  This can have significant performance implications including some performance degradations in comparison to prior SQLAlchemy versions.  Set this attribute to True if this object can make use of the cache key generated by the superclass.  Alternatively, this attribute may be set to False which will disable this warning. (Background on this error at: https://sqlalche.me/e/14/cprf)
  return self._connection.execute(select_query)
/opt/conda/lib/python3.9/site-packages/datacube/drivers/postgres/_api.py:564: SAWarning: Class Float8Range will not make use of SQL compilation caching as it does not set the 'inherit_cache' attribute to ``True``.  This can have significant performance implications including some performance degradations in comparison to prior SQLAlchemy versions.  

<xarray.Dataset>
Dimensions:      (time: 21, y: 1031, x: 1010)
Coordinates:
  * time         (time) datetime64[ns] 2020-10-01T08:28:17 ... 2020-12-30T08:...
  * y            (y) float64 6.807e+06 6.807e+06 ... 6.797e+06 6.797e+06
  * x            (x) float64 8.687e+05 8.687e+05 ... 8.787e+05 8.788e+05
    spatial_ref  int32 32734
Data variables:
    blue         (time, y, x) uint16 800 955 1188 1404 ... 2574 2502 2552 2722
    green        (time, y, x) uint16 856 1044 1222 1444 ... 2378 2508 2426 2432
    red          (time, y, x) uint16 927 1154 1326 1552 ... 2402 2332 2290 2348
    nir          (time, y, x) uint16 1128 1324 1572 1824 ... 3286 3348 3378 3398
Attributes:
    crs:           EPSG:32734
    grid_mapping:  spatial_ref

<a id='index_array3'></a>
## **Advanced Indexing**
### 1) Temporal Subset

In the earlier tutorial, we introduced `isel()`and `sel()` for indexing data. For both methods, a **slicing** operator exists. If the function `slice()` is passed onto the index function, the dataset is sliced. 
The first example uses the slicing by position method to select the first five scenes in `ds`. The start value is included (here, 0) and the stop value (here, 5) is excluded.

#### I. Using index number

In [9]:
ds.isel(time=slice(0,5))
#ds.isel(time = [0,1,2,3,4])

<xarray.Dataset>
Dimensions:      (time: 5, y: 1031, x: 1010)
Coordinates:
  * time         (time) datetime64[ns] 2020-10-01T08:28:17 ... 2020-10-26T08:...
  * y            (y) float64 6.807e+06 6.807e+06 ... 6.797e+06 6.797e+06
  * x            (x) float64 8.687e+05 8.687e+05 ... 8.787e+05 8.788e+05
    spatial_ref  int32 32734
Data variables:
    blue         (time, y, x) uint16 800 955 1188 1404 1584 ... 828 719 730 815
    green        (time, y, x) uint16 856 1044 1222 1444 ... 1154 983 930 1166
    red          (time, y, x) uint16 927 1154 1326 1552 ... 1648 1306 1414 1630
    nir          (time, y, x) uint16 1128 1324 1572 1824 ... 2254 2128 2208 2372
Attributes:
    crs:           EPSG:32734
    grid_mapping:  spatial_ref

In [10]:
ds.isel(time=slice(0,5)).time

<xarray.DataArray 'time' (time: 5)>
array(['2020-10-01T08:28:17.000000000', '2020-10-11T08:28:18.000000000',
       '2020-10-16T08:28:20.000000000', '2020-10-21T08:28:18.000000000',
       '2020-10-26T08:28:19.000000000'], dtype='datetime64[ns]')
Coordinates:
  * time         (time) datetime64[ns] 2020-10-01T08:28:17 ... 2020-10-26T08:...
    spatial_ref  int32 32734
Attributes:
    units:    seconds since 1970-01-01 00:00:00

#### II. Using `datetime64` data

This example uses the slicing by label method to select the scenes between "2020-12-01" and "2020-12-15". Note, that when using the `slice()` function with the `sel()` method, both start and stop value are included.

In [14]:
print(ds.sel(time=slice("2020-12-01","2020-12-15"))) 

#ds.sel(time=slice("2018-01-01", "2018-01-31")) # wrong time frame

<xarray.Dataset>
Dimensions:      (time: 6, y: 1031, x: 1010)
Coordinates:
  * time         (time) datetime64[ns] 2020-12-03T08:38:08 ... 2020-12-15T08:...
  * y            (y) float64 6.807e+06 6.807e+06 ... 6.797e+06 6.797e+06
  * x            (x) float64 8.687e+05 8.687e+05 ... 8.787e+05 8.788e+05
    spatial_ref  int32 32734
Data variables:
    blue         (time, y, x) uint16 724 713 706 699 704 ... 611 641 452 507 671
    green        (time, y, x) uint16 1004 988 985 996 991 ... 929 730 813 984
    red          (time, y, x) uint16 1192 1184 1200 1184 ... 1218 1038 1078 1400
    nir          (time, y, x) uint16 2454 2464 2332 2316 ... 2198 2254 2388 2414
Attributes:
    crs:           EPSG:32734
    grid_mapping:  spatial_ref


In [15]:
ds.sel(time=slice("2020-12-01","2020-12-15")).time

<xarray.DataArray 'time' (time: 6)>
array(['2020-12-03T08:38:08.000000000', '2020-12-05T08:28:12.000000000',
       '2020-12-08T08:38:07.000000000', '2020-12-10T08:28:11.000000000',
       '2020-12-13T08:38:06.000000000', '2020-12-15T08:28:12.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time         (time) datetime64[ns] 2020-12-03T08:38:08 ... 2020-12-15T08:...
    spatial_ref  int32 32734
Attributes:
    units:    seconds since 1970-01-01 00:00:00

#### III. Using other time dimensions

`xarray` also includes some useful features for the inspection of the time dimension. It helps extract additional information from a dataset efficiently. The following code automatically groups the time dimension in seasons ("DJF", "MAM", JJA", "SON"). There are a lot of other `time` dimensions arguments, e.g., `month`, `week`, `weekday`, `dayofyear`.

In [27]:
ds.time.dt.season

<xarray.DataArray 'season' (time: 21)>
array(['SON', 'SON', 'SON', 'SON', 'SON', 'SON', 'SON', 'SON', 'SON',
       'SON', 'SON', 'SON', 'SON', 'DJF', 'DJF', 'DJF', 'DJF', 'DJF',
       'DJF', 'DJF', 'DJF'], dtype='<U3')
Coordinates:
  * time         (time) datetime64[ns] 2020-10-01T08:28:17 ... 2020-12-30T08:...
    spatial_ref  int32 32734

In [28]:
ds.time.dt.month

<xarray.DataArray 'month' (time: 21)>
array([10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12,
       12, 12, 12, 12])
Coordinates:
  * time         (time) datetime64[ns] 2020-10-01T08:28:17 ... 2020-12-30T08:...
    spatial_ref  int32 32734

In [29]:
ds.time.dt.weekday

<xarray.DataArray 'dayofweek' (time: 21)>
array([3, 6, 4, 2, 0, 5, 3, 1, 4, 6, 4, 2, 0, 3, 5, 1, 3, 6, 1, 6, 2])
Coordinates:
  * time         (time) datetime64[ns] 2020-10-01T08:28:17 ... 2020-12-30T08:...
    spatial_ref  int32 32734

It is also possible to extract the "day of year" for a time step.

In [30]:
ds.time.dt.dayofyear

<xarray.DataArray 'dayofyear' (time: 21)>
array([275, 285, 290, 295, 300, 305, 310, 315, 318, 320, 325, 330, 335,
       338, 340, 343, 345, 348, 350, 355, 365])
Coordinates:
  * time         (time) datetime64[ns] 2020-10-01T08:28:17 ... 2020-12-30T08:...
    spatial_ref  int32 32734

In [31]:
ds.groupby('time.season')

DatasetGroupBy, grouped over 'season'
2 groups with labels 'DJF', 'SON'.

In [25]:
#ds.groupby('time.season').mean()

<bound method DatasetGroupByReductions.mean of DatasetGroupBy, grouped over 'season'
2 groups with labels 'DJF', 'SON'.>

### 2) Spatial Subset
It is possible to index and **slice within the x and y dimensions**. The following example selects the value for pixels of all bands in the second column and the fifth row of the raster (`x=2,y=5`).

In [32]:
ds.isel(x=2, y=5)
#ds.isel(x=[0,1,2], y=5)

<xarray.Dataset>
Dimensions:      (time: 21)
Coordinates:
  * time         (time) datetime64[ns] 2020-10-01T08:28:17 ... 2020-12-30T08:...
    y            float64 6.807e+06
    x            float64 8.687e+05
    spatial_ref  int32 32734
Data variables:
    blue         (time) uint16 1836 1012 1032 1282 1202 ... 530 650 785 11488
    green        (time) uint16 1920 1382 1444 1654 1632 ... 815 1020 1104 10920
    red          (time) uint16 2126 1990 2042 2242 2204 ... 973 1260 1376 10720
    nir          (time) uint16 2356 2820 2764 3096 2840 ... 2112 2306 2358 9824
Attributes:
    crs:           EPSG:32734
    grid_mapping:  spatial_ref

### 3) Combining Temporal and Spatial Subset

We can subset temporally and spatially using `slice()` operator. If you know the actual coordinate (x,y) value (extent) of the spatial subset area, use the `sel()` function.

The following example subsets the `ds` by the temporal and spatial location of the pixels. Only the pixels from the first to the fifth columns and the pixels from the first to the fifth rows are included in the output. Also, the scenes are filtered in the time dimension between the first and fifth time step.

In [34]:
ds2 = ds.isel(time=slice(0,5), x= slice(0,5), y=slice(0,5))
ds2

#ds2.time
#plt.scatter(ds2.x.values, ds2.y.values)

<xarray.Dataset>
Dimensions:      (time: 5, y: 5, x: 5)
Coordinates:
  * time         (time) datetime64[ns] 2020-10-01T08:28:17 ... 2020-10-26T08:...
  * y            (y) float64 6.807e+06 6.807e+06 6.807e+06 6.807e+06 6.807e+06
  * x            (x) float64 8.687e+05 8.687e+05 8.687e+05 8.687e+05 8.687e+05
    spatial_ref  int32 32734
Data variables:
    blue         (time, y, x) uint16 800 955 1188 1404 ... 1272 1208 1180 1260
    green        (time, y, x) uint16 856 1044 1222 1444 ... 1632 1568 1576 1630
    red          (time, y, x) uint16 927 1154 1326 1552 ... 2250 2176 2164 2230
    nir          (time, y, x) uint16 1128 1324 1572 1824 ... 2890 2840 2786 2854
Attributes:
    crs:           EPSG:32734
    grid_mapping:  spatial_ref

## **Data Manipulation & Statistics**

This notebook presents some basic built-in functions of the `xarray` library to manipulate and transform data in a `xarray.Dataset`. Here, we show only a fraction of the available `xarray` functions. For a complete overview of all the available functions and tools of the `xarray` package, please visit the [documentation website](http://xarray.pydata.org/en/stable/). 

[Notebook 07](07_basic_analysis.ipynb) will cover this topic, focusing on an application-oriented remote sensing approach.
###  1) Statistical Operation

The simple built-in functions allow the user to do simple calculations with a `xarray.Dataset`.
The **basic math** built-in `xarray` functions are:
* `min()`, `max()`
* `mean()`, `median()`
* `sum()`
* `std()`

The following code demonstrates the easy use of the `max()` function to extract the maximum value of the red band in the `ds` dataset.

In [36]:
print(ds.red.max())

<xarray.DataArray 'red' ()>
array(16736, dtype=uint16)
Coordinates:
    spatial_ref  int32 32734


To apply a function to every value of a specified dimension (e.g., to calculate the mean of every time step), the `dim` argument in the basic math function must be defined with the dimension label.

This example calculates the mean of the `red` band for each pixel (defined by the unique `x`, `y` combination) over every time step. The result is a data array that can be used for further time series visualization and analysis.

In [37]:
print(ds.red.mean(dim=["x", "y"]))

#ds.red.mean(dim=["x", "y"]).values
#plt.plot(ds.red.mean(dim=["x", "y"]).values)

<xarray.DataArray 'red' (time: 21)>
array([2.77713346e+03, 1.95538800e+03, 2.02523178e+03, 2.06630744e+03,
       2.06889537e+03, 9.36111112e+03, 8.58714741e+03, 1.58639660e+03,
       5.92408889e+00, 1.62720067e+03, 1.41386830e+03, 1.33785390e+03,
       1.63493247e+03, 6.85420576e+00, 1.71084498e+03, 8.81949775e+01,
       1.38447213e+03, 1.59972900e+01, 1.48100963e+03, 1.58694061e+03,
       4.93148212e+03])
Coordinates:
  * time         (time) datetime64[ns] 2020-10-01T08:28:17 ... 2020-12-30T08:...
    spatial_ref  int32 32734


This examples works the other way around. It calculates the standard deviation of every pixel (`x`, `y`) over all timesteps of the dataset `ds`.

In [38]:
print(ds.red.std(dim="time"))

<xarray.DataArray 'red' (y: 1031, x: 1010)>
array([[3670.58587698, 3674.904827  , 3660.70321733, ..., 2949.90060481,
        2956.18840206, 2972.75273914],
       [3677.38257257, 3660.0924068 , 3635.19863746, ..., 2978.732251  ,
        2977.08159459, 2982.44784944],
       [3640.23745457, 3626.66965036, 3611.31677796, ..., 3014.86105846,
        2993.94438258, 2978.65476949],
       ...,
       [2858.49614935, 2890.04706315, 2914.90605287, ..., 3044.32230319,
        3057.25457146, 3021.09709315],
       [2866.49764096, 2887.5721318 , 2909.42651228, ..., 3080.21076229,
        3072.09812709, 3010.72221167],
       [2934.69442448, 2908.81022814, 2917.17634419, ..., 3098.16155785,
        3086.02240525, 3016.62641083]])
Coordinates:
  * y            (y) float64 6.807e+06 6.807e+06 ... 6.797e+06 6.797e+06
  * x            (x) float64 8.687e+05 8.687e+05 ... 8.787e+05 8.788e+05
    spatial_ref  int32 32734


Remember, to access the raw `numpy` array that stores the values of the resulting `xarray.DataArrays`, the suffix `.values` is needed. This allows you to work with the "actual" data values.

In [39]:
print(ds.blue.sum(dim=["x","y"]).values)
#plt.plot(ds.blue.sum(dim=["x","y"]).values)

[ 2240130044   983668505  1001300481  1215208047  1137865078 11100670146
  9956673237   872160019     3758740   906014035   816555416   728884192
  1194521042     4236264  1136981253   108670046   913016514     9934090
   800405132   923820606  5631112474]


### 2) Conditional Operation

Using conditional operation can be very helpful when we need to analyze satellite scenes or pixels that lie within our interests. The `where()` function provides the option to **mask** a `xarray.Dataset` based on a logical condition. By default, the function converts all values that match the condition to NaN values. This is extremely useful when applied with a binary mask to mask your data to the desired values. The argument `other` lets you define a subset value for all values that match the condition (default is `nan`). The argument `drop` drops all values which do not correspond with the condition.
The following example masks the dataset `ds` to only the values with a reflectance value greater than 700 in the `red` band.

In [40]:
print(ds.where(ds.red > 700))
#print(ds.where(ds.red < 700))

<xarray.Dataset>
Dimensions:      (time: 21, y: 1031, x: 1010)
Coordinates:
  * time         (time) datetime64[ns] 2020-10-01T08:28:17 ... 2020-12-30T08:...
  * y            (y) float64 6.807e+06 6.807e+06 ... 6.797e+06 6.797e+06
  * x            (x) float64 8.687e+05 8.687e+05 ... 8.787e+05 8.788e+05
    spatial_ref  int32 32734
Data variables:
    blue         (time, y, x) float64 800.0 955.0 ... 2.552e+03 2.722e+03
    green        (time, y, x) float64 856.0 1.044e+03 ... 2.426e+03 2.432e+03
    red          (time, y, x) float64 927.0 1.154e+03 ... 2.29e+03 2.348e+03
    nir          (time, y, x) float64 1.128e+03 1.324e+03 ... 3.398e+03
Attributes:
    crs:           EPSG:32734
    grid_mapping:  spatial_ref


This code subsets all zeros in the red band of the dataset `ds` in the first time stamp with the new value -9999.

In [41]:
replace = ds.red.isel(time=0).where(ds.red != 0, other = -9999)
#replace.values.min()

The implemented `xarray` function `isin()` allows us to **test each value** of `xarray.Dataset` or `xarray.DataArray` whether it is in the elements defined within the function. It returns a boolean array which can be used as a mask.
This example checks all the values of the `red` measurement if the value is in an array from 0 to 550.

In [42]:
mask_red = ds.red.isin(range(550))
print(mask_red)

#plt.imshow(mask_red) #error
#plt.imshow(mask_red.isel(time=3))

<xarray.DataArray 'red' (time: 21, y: 1031, x: 1010)>
array([[[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
...
        ...,
        [False, False, False, .

The created mask can easily be combined with the `where()` function to filter the dataset based on the predefined mask. In this case, the `ds` dataset is masked with previously defined mask `mask_red`, which is based on a logical test if values of the `red` band are within a specific range of values.

In [43]:
print(ds.where(mask_red)) #masking

<xarray.Dataset>
Dimensions:      (time: 21, y: 1031, x: 1010)
Coordinates:
  * time         (time) datetime64[ns] 2020-10-01T08:28:17 ... 2020-12-30T08:...
  * y            (y) float64 6.807e+06 6.807e+06 ... 6.797e+06 6.797e+06
  * x            (x) float64 8.687e+05 8.687e+05 ... 8.787e+05 8.788e+05
    spatial_ref  int32 32734
Data variables:
    blue         (time, y, x) float64 nan nan nan nan nan ... nan nan nan nan
    green        (time, y, x) float64 nan nan nan nan nan ... nan nan nan nan
    red          (time, y, x) float64 nan nan nan nan nan ... nan nan nan nan
    nir          (time, y, x) float64 nan nan nan nan nan ... nan nan nan nan
Attributes:
    crs:           EPSG:32734
    grid_mapping:  spatial_ref


### 3) Resampling
Resampling is necessary when working with time-series data if we want the data product to align with the temporal window.

 - **resample()**

The **`resample()` method** allows us to summarise the `xarray.Dataset` into bigger or smaller chunks based on a dimension. It handles both upsampling and downsampling. The argument `time` needs to be defined as a datetime-like coordinate. In the following example, we resample the `ds` dataset to a monthly time interval (`time = "m"`) and then calculate the median value for every resample chunk. _(this process takes a little while to run)_

In [44]:
print(ds.resample(time='m').median())

<xarray.Dataset>
Dimensions:      (time: 3, y: 1031, x: 1010)
Coordinates:
  * time         (time) datetime64[ns] 2020-10-31 2020-11-30 2020-12-31
  * y            (y) float64 6.807e+06 6.807e+06 ... 6.797e+06 6.797e+06
  * x            (x) float64 8.687e+05 8.687e+05 ... 8.787e+05 8.788e+05
    spatial_ref  int32 32734
Data variables:
    blue         (time, y, x) float64 1.035e+03 1.055e+03 ... 576.0 722.5
    green        (time, y, x) float64 1.47e+03 1.456e+03 ... 871.5 1.032e+03
    red          (time, y, x) float64 2.057e+03 2.04e+03 ... 1.061e+03 1.377e+03
    nir          (time, y, x) float64 2.82e+03 2.79e+03 ... 2.425e+03 2.455e+03


 - **groupby() method**

The **`groupby()` method** can also be used within the `xarray` library to *aggregate data over time*. Time aggregation arguments can be e.g. "time.year", "time.season", "time.month", "time.week", "time.day".
The code below groups the `ds` dataset into two groups by year. Therefore, a new "dimension" `year` is created. Then the median for each `year` is calculated. _(this process takes a little while to run)_

In [20]:
print(ds.groupby("time.year").median(dim="time"))

<xarray.Dataset>
Dimensions:      (y: 1031, x: 1010, year: 1)
Coordinates:
  * y            (y) float64 6.807e+06 6.807e+06 ... 6.797e+06 6.797e+06
  * x            (x) float64 8.687e+05 8.687e+05 ... 8.787e+05 8.788e+05
    spatial_ref  int32 32734
  * year         (year) int64 2020
Data variables:
    blue         (year, y, x) float64 800.0 810.0 820.0 ... 541.0 584.0 725.0
    green        (year, y, x) float64 1.184e+03 1.144e+03 ... 860.0 1.026e+03
    red          (year, y, x) float64 1.39e+03 1.35e+03 ... 1.158e+03 1.438e+03
    nir          (year, y, x) float64 2.71e+03 2.646e+03 ... 2.292e+03 2.414e+03


### 4) Interpolation
Interpolation is a common solution dealing with missing remote sensing data, either caused by the coarse temporal resolution of the satellite, high cloud cover, or bad quality of the scenes. For example, a scene of a specific date is not available in the dataset. With the implemented `interp()`, it is possible to **interpolate data** for predefined time steps. The function takes the next usable scene before and after the specified date and interpolates their values (by default, interpolation method is "linear") to build a new `xarray.Dataset`.

In this example, the `ds` dataset has missing scenes on the "2020-12-25". The `interp()` function builds a "new" scene based on a linear interpolation from the two measurements before and after the new time step.

In [46]:
print(ds.time)

<xarray.DataArray 'time' (time: 21)>
array(['2020-10-01T08:28:17.000000000', '2020-10-11T08:28:18.000000000',
       '2020-10-16T08:28:20.000000000', '2020-10-21T08:28:18.000000000',
       '2020-10-26T08:28:19.000000000', '2020-10-31T08:28:17.000000000',
       '2020-11-05T08:28:19.000000000', '2020-11-10T08:28:15.000000000',
       '2020-11-13T08:38:10.000000000', '2020-11-15T08:28:17.000000000',
       '2020-11-20T08:28:15.000000000', '2020-11-25T08:28:15.000000000',
       '2020-11-30T08:28:14.000000000', '2020-12-03T08:38:08.000000000',
       '2020-12-05T08:28:12.000000000', '2020-12-08T08:38:07.000000000',
       '2020-12-10T08:28:11.000000000', '2020-12-13T08:38:06.000000000',
       '2020-12-15T08:28:12.000000000', '2020-12-20T08:28:10.000000000',
       '2020-12-30T08:28:12.000000000'], dtype='datetime64[ns]')
Coordinates:
  * time         (time) datetime64[ns] 2020-10-01T08:28:17 ... 2020-12-30T08:...
    spatial_ref  int32 32734
Attributes:
    units:    seconds since 1970-

In [47]:
ds_interp = ds.interp(time=["2020-12-25"])
print(ds_interp)

<xarray.Dataset>
Dimensions:      (y: 1031, x: 1010, time: 1)
Coordinates:
  * y            (y) float64 6.807e+06 6.807e+06 ... 6.797e+06 6.797e+06
  * x            (x) float64 8.687e+05 8.687e+05 ... 8.787e+05 8.788e+05
    spatial_ref  int32 32734
  * time         (time) datetime64[ns] 2020-12-25
Data variables:
    blue         (time, y, x) float64 5.935e+03 6.006e+03 ... 1.695e+03
    green        (time, y, x) float64 5.733e+03 5.686e+03 ... 1.725e+03
    red          (time, y, x) float64 5.465e+03 5.501e+03 ... 1.702e+03 1.91e+03
    nir          (time, y, x) float64 5.97e+03 5.941e+03 ... 2.888e+03 2.915e+03
Attributes:
    crs:           EPSG:32734
    grid_mapping:  spatial_ref


The `merge()` function allows us to **merge/join** `xarray.Datasets` or variables. By default, the `merge()` function uses an "inner" join as a merging operation. 
In our example, the interpolated `xarray.Dataset` created above is merged to the `ds` dataset using the `merge()` function.

In [48]:
print(ds.merge(ds_interp).time)

<xarray.DataArray 'time' (time: 22)>
array(['2020-10-01T08:28:17.000000000', '2020-10-11T08:28:18.000000000',
       '2020-10-16T08:28:20.000000000', '2020-10-21T08:28:18.000000000',
       '2020-10-26T08:28:19.000000000', '2020-10-31T08:28:17.000000000',
       '2020-11-05T08:28:19.000000000', '2020-11-10T08:28:15.000000000',
       '2020-11-13T08:38:10.000000000', '2020-11-15T08:28:17.000000000',
       '2020-11-20T08:28:15.000000000', '2020-11-25T08:28:15.000000000',
       '2020-11-30T08:28:14.000000000', '2020-12-03T08:38:08.000000000',
       '2020-12-05T08:28:12.000000000', '2020-12-08T08:38:07.000000000',
       '2020-12-10T08:28:11.000000000', '2020-12-13T08:38:06.000000000',
       '2020-12-15T08:28:12.000000000', '2020-12-20T08:28:10.000000000',
       '2020-12-25T00:00:00.000000000', '2020-12-30T08:28:12.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time         (time) datetime64[ns] 2020-10-01T08:28:17 ... 2020-12-30T08:...
    spatial_ref  int32 32734
Attrib

The `xarray` package contains a variety of other useful functions besides those shown here. For more information about the `xarray` package, visit the [documentation website](http://xarray.pydata.org/en/stable/).

The `xarray.Datasets` in the eo2cube datacube environment are a useful and effective structure for handling remote sensing raster data. In this notebook, you learned the basic structure and application methods of `xarray.Datasets`and `xarray.DataArrays`. The following [notebook 06](06_plotting.ipynb) will cover how to visualize xarray raster data nicely and efficiently.

## Recommended next steps

To continue working through the notebooks in this beginner's guide, the following notebooks are designed to be worked through in the following order:

1. [Jupyter Notebooks](https://github.com/eo2cube/eo2cube_notebooks/blob/main/get_started/intro_to_eo2cube/01_jupyter_introduction.ipynb)
2. [eo2cube](https://github.com/eo2cube/eo2cube_notebooks/blob/main/get_started/intro_to_eo2cube/02_eo2cube_introduction.ipynb)
3. [Loading Data](https://github.com/eo2cube/eo2cube_notebooks/blob/main/get_started/intro_to_eo2cube/03_data_lookup_and_loading.ipynb)
4. [Xarray I: Data Structure](https://github.com/eo2cube/eo2cube_notebooks/blob/main/get_started/intro_to_eo2cube/04_xarrayI_data_structure.ipynb)
5. ***Xarray II: Index and Statistics (this notebook)***
6. [Plotting data](https://github.com/eo2cube/eo2cube_notebooks/blob/main/get_started/intro_to_eo2cube/06_plotting_basics.ipynb)
7. [Spatial analysis](https://github.com/eo2cube/eo2cube_notebooks/blob/main/get_started/intro_to_eo2cube/07_basic_analysis.ipynb)
8. [Parallel processing with Dask](https://github.com/eo2cube/eo2cube_notebooks/blob/main/get_started/intro_to_eo2cube/08_parallel_processing_with_dask.ipynb)

The additional notebooks are designed for users to build up both basic and advanced skills which are not covered by the beginner's guide. Self-motivated users can go through them according to their own needs. They act as complements for the guide:
<br>

1. [Python's file management tools](https://github.com/eo2cube/eo2cube_notebooks/blob/main/get_started/intro_to_eo2cube/I_file_management.ipynb)
2. [Image Processing basics using NumPy and Matplotlib](https://github.com/eo2cube/eo2cube_notebooks/blob/main/get_started/intro_to_eo2cube/II_numpy_image_processing.ipynb)
3. [Vector Processing](https://github.com/eo2cube/eo2cube_notebooks/blob/main/get_started/intro_to_eo2cube/III_process_vector_data.ipynb)
4. [Advanced Plotting](https://github.com/eo2cube/eo2cube_notebooks/blob/main/get_started/intro_to_eo2cube/IV_advanced_plotting.ipynb)

***

## Additional information

<font size="2">This notebook for the usage in the Open Data Cube entities of the [Department of Remote Sensing](http://remote-sensing.org/), [University of Wuerzburg](https://www.uni-wuerzburg.de/startseite/), is adapted from [Geoscience Australia](https://github.com/GeoscienceAustralia/dea-notebooks), published using the Apache License, Version 2.0. Thanks! </font>

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Australia data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.


**Contact:** If you would like to report an issue with this notebook, you can file one on [Github](https://github.com).

**Last modified:** May 2022